### Performance reports

#### Vanila insert and commit
- Block heights 0 to 621100: About 2 days (48 Hours)

In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is running!')

In [ ]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [ ]:
# preparation database
import sqlite3

db_path = 'databases.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()

def create_txhash_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxHash (
                     txhash TEXT PRIMARY KEY,
                     height INTEGER NOT NULL);
                ''')

def insert_txhash(txhash, height):
    cur.execute(f'''INSERT OR REPLACE INTO TxHash (
                   txhash, height) VALUES (
                   "{txhash}", {height});
                ''')
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    
def insert_txhash_many(txhash, height):
    cur.executemany('''INSERT OR REPLACE INTO TxHash (
                       txhash, height) VALUES (
                       ?, ?});
                    ''', (txhash, height))

def get_max_height():
    cur.execute('''SELECT MAX(height) FROM TxHash;''')
    return cur.fetchone()[0]

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

In [ ]:
%%time
import time

create_txhash_table()
conn.commit()

start_height = get_max_height()
if start_height is None:
    start_height = 0
print(f'Start from {start_height}')

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)

begin_transactions()
time1 = time.time()
for i in range(start_height, best_block["height"]-99):
    block_hash = rpc_connection.getblockhash(i)
    block = rpc_connection.getblock(block_hash)
    height = block['height']
    for t in block['tx']:
        insert_txhash(t, height)
    if i % 1000 == 0:
        commit_transactions()
        begin_transactions()
        time2 = time.time()
        print(f'[{round(time2-time1)}] Reading {i} / {best_block["height"]} ({i/best_block["height"]*100}%)',
              end='\r')
        time1 = time2
commit_transactions()
conn.close()
print('')
print(f'Query-reply done.')
print(f'txhash heights: {i}')